In [88]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import logging
import pandas as pd
import re
import time
import json

In [89]:
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [90]:
driver = webdriver.Chrome(service=Service(executable_path="/opt/homebrew/bin/chromedriver"), options=options)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="4301bfa279d448ad58f4bc8a34f4cb7c")>

In [91]:
table_rows = driver.find_elements(By.TAG_NAME, 'tr')
type(table_rows)
len(table_rows)
# table_rows


16

In [93]:
def get_all_table_rows_except_header_row():
    table_rows = driver.find_elements(By.TAG_NAME, 'tr')
    
    return table_rows[1:]

table_rows = get_all_table_rows_except_header_row()
len(table_rows)

15

In [94]:
def wait_for_element(locator, locator_type, timeout):
    try:
        WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located((locator_type, locator)))
        return True
    except Exception as e:
        logging.exception(f'An unexpected error occurred: {e}')
        return False

In [95]:
def wait_for_and_find_element(locator, locator_type, timeout):
    try:
        wait = wait_for_element(locator, locator_type, timeout)
        if not wait:
            logging.error(f'Tried to wait to locate element via locator "{locator}", but timed out')
            return False, None
        element = driver.find_element(locator_type, locator)
        if not element:
            logging.error(f'Could not find element via locator "{locator}"')
            return True, None
        if wait and element:
            logging.info(f'Found and located element via locator "{locator}"')
            return True, element

    except Exception as NoSuchElementException:
        logging.exception(f'An unexpected error occurred: {NoSuchElementException}')


In [96]:
def find_element_and_click(locator ,locator_type=By.CSS_SELECTOR):
    """
    Finds element and clicks it using `WebElement.click()`
    :param locator:
    :param locator_type:
    :return: Tuple(bool, WebElement)
    """
    try:
        element = driver.find_element(locator_type, locator)
        if not element:
            logging.error(f'Could not locate element via locator "{locator}".')
        element.click()
        return True, element
    except NoSuchElementException:
        logging.exception(f'Element {locator} was not found.')
        return False, None
    except Exception as e:
        logging.exception(f'Error occurred when trying to find and click element with locator: "{locator}" resulting in error message: {str(e)}')
        return False, None


In [97]:
def find_and_click_exit_button():
    exit_btn_clicked, _ = find_element_and_click(
        locator='//*[@id="MerchantApp"]/div/div/div[3]/div[2]/div[2]/div/div/div[1]/nav/div[1]/div[1]/div/button',
        locator_type=By.XPATH)
    return exit_btn_clicked


In [98]:
def process_row(table_row, orders):
    table_row.click()
    time.sleep(5)  # wait for ssb to load on dom
    
    found, elem = wait_for_and_find_element(locator="//*[@class='styles__SidesheetContent-sc-czzuxh-2 hKVVOI']", locator_type=By.XPATH, timeout=10)


    if found:
        start_length = len(orders)
        orders.append(elem.text)
        end_length = len(orders)

        if end_length > start_length:
            exit_btn_clicked = find_and_click_exit_button()

            if exit_btn_clicked:
                logging.info(f'Exiting sidesheetbody.')
                return True
    return False


In [99]:
def iterate_table_rows(table_rows, orders):
    idx = 0 
    while idx < len(table_rows):
        table_row = table_rows[idx]

        if process_row(table_row, orders):
            idx += 1
        else:
            logging.error(f"Failed to process Order #: {idx + 1}")



In [100]:
def orders_scraper():
    orders = []
    table_rows = get_all_table_rows_except_header_row()
    
    iterate_table_rows(table_rows, orders)
    
    return orders
    

In [101]:
# def orders_scraper():
#     orders = []
#     
#     for idx, table_row in enumerate(table_rows):
#         if idx >= 1:
#             table_row.click()
#             
#             time.sleep(5)  # wait for ssb to load on dom
#             
#             found, elem = wait_for_and_find_element(locator="//*[@class='styles__SidesheetContent-sc-czzuxh-2 hKVVOI']", locator_type=By.XPATH, timeout=10)
#             
#             
#             start_length = len(orders)
#             orders.append(elem.text)
#             end_length = len(orders)
#             
#             if end_length > start_length:
#                 exit_btn_clicked = find_element_and_click(locator='//*[@id="MerchantApp"]/div/div/div[3]/div[2]/div[2]/div/div/div[1]/nav/div[1]/div[1]/div/button', locator_type=By.XPATH)
# 
#                 if exit_btn_clicked:
#                     logging.info(f'Exiting sidesheetbody for Order #: {idx}')
# 
#     return orders


In [102]:
# orders = orders_scraper()
# orders

In [103]:
def clean_order_text(order):
    # Define the patterns in a list
    patterns = [
        re.compile(r'(Rate Dasher|Learn More|Get help).*?(?=Pick Up Location)', re.DOTALL),
        re.compile(r'Channel.*?(?=Order Details)', re.DOTALL),
        re.compile(r'Associated Transactions \(\d+\).*?(?=Transaction #\d+ - Delivery)', re.DOTALL),
        re.compile(r'Associated Transactions \(\d+\).*?(?=Transaction #\d+ - Pickup)', re.DOTALL)

    ]

    # Apply each pattern
    for pattern in patterns:
        order = re.sub(pattern, '', order).strip()

    # Replace newline characters with spaces
    order = order.replace('\n', ' ')

    return order


In [104]:
def parse_menu_items(price_as_value):
    # Initialize a dictionary for the nested order details
    item_name_to_item_price = {}

    # Regular expression pattern to match an item and its corresponding price
    pattern = re.compile(r'(.+?) (\$\d+\.\d+)')

    # Find all matches in the string
    matches = pattern.findall(price_as_value)

    # Loop through all the matches and populate the dictionary
    for item_name, item_price in matches:
        item_name_to_item_price[item_name] = item_price

    return item_name_to_item_price


In [105]:
def parse_order(order):
    # Define the known keys with their regex patterns
    known_keys = [r'\bOrder: \b', r'\bDelivered\b', r'\bCustomer\sPicked\sUp\b', r'\bCancelled\s-\sNot\sPaid\b', r'\bCancelled\s-\sPaid\b', r'\bPick Up Location\b', r'\bOrder Details\b',
                  r'\bSubtotal\b(?=\s[^a-zA-Z])', r'\bSubtotal\sTax\b', r'Commission \(\d+%\)',
                  r'\bTotal Customer Refund\b', r'\bEstimated Payout\b', r'\bAssociated Transactions \(\d+%\)',
                  r'Transaction #\d+ - Delivery']

    # Initialize the dictionary to store our parsed values
    order_dict = {}

    # Find positions of known keys
    positions = []
    for key in known_keys:
        for m in re.finditer(key, order):
            positions.append((m.start(), m.end(), m.group()))

    # Sort positions by their start index
    positions.sort(key=lambda x: x[0])
    # print(positions)

    # Create segments based on positions
    segments = []

    for i in range(len(positions)):
        start = positions[i][1]
        end = positions[i + 1][0] if i + 1 < len(positions) else len(order)
        key = positions[i][2]
        value = order[start:end].strip().split(' ')[0] if 'Estimated Payout' in key else order[start:end].strip()
        segments.append((key, value))

    # Parse segments into dictionary
    for key, value in segments:
        if key == "Order Details":
            value = parse_menu_items(value)
        order_dict[key.replace(':', '').strip()] = value

    return order_dict


In [108]:
def main_looper_func(orders):

    results = []

    for order in orders:
        # 1. Get order strings squeaky clean!
        order_cleaned = clean_order_text(order)

        # 2. Parse each cleaned order string into formatted key/value pairs
        parsed_order = parse_order(order_cleaned)

        results.append(parsed_order)

    return results


In [109]:
orders = orders_scraper()

In [38]:
results = main_looper_func(orders)

In [41]:
def prettify_list_of_dicts(list_of_dicts):
    return json.dumps(list_of_dicts, indent=4)


print(prettify_list_of_dicts(results))



[
    {
        "Order": "E4D52343",
        "Delivered": "The order was delivered at 11:20 PM on September 27, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "1 \u00d7 Hostess Chocolate Ding Dong 2pk (2.55oz) (Pastry TX)": "$3.79",
            " 1 \u00d7 Coke Cherry Bottle (20oz) (Soda TX)": "$3.79",
            " 1 \u00d7 Coke Classic Bottle (20oz) (Soda TX)": "$3.19"
        },
        "Subtotal": "$10.77",
        "Subtotal Tax": "$0.89",
        "Commission (16%)": "-$1.72",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$9.94",
        "Transaction #8079836740 - Delivery": "$9.94"
    },
    {
        "Order": "28F4404A",
        "Delivered": "The order was delivered at 10:46 PM on September 27, 2023.",
        "Pick Up Location": "800 W Interstate 20, Big Spring, TX 79720, USA",
        "Order Details": {
            "1 \u00d7 Funyuns Flavored Onion Rings (2.125 oz) (Chips & Popcorn

In [17]:
def get_order_id(order_text):
    # Extract the order_id
    order_id_start = order_text.find("Order:") + len("Order:")
    order_id_end = order_text.find("\n", order_id_start)
    order_id = order_text[order_id_start:order_id_end]
    return order_id

In [92]:
def orders_spreadsheet_creator(orders):
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
        
        # Loop through order_contents and create a sheet for each order
        for idx, order_text in enumerate(orders, start=1):
            
            # get order id
            order_id = get_order_id(order_text)
            
            # Convert the order text to a DataFrame
            df = pd.DataFrame(orders, columns=[f'{order_id}'])
            
            # Write the DataFrame to the Excel sheet with the order ID as the sheet name
            # TODO: store num getter func 
            df.to_excel(writer, sheet_name=f"store_num", index=False)
            
    # Close the Pandas Excel writer and save the file
    writer.save() #todo: AttributeError: 'XlsxWriter' object has no attribute 'save'


orders_spreadsheet_creator(orders_content)

AttributeError: 'XlsxWriter' object has no attribute 'save'

In [98]:
def create_excel_from_orders(order_strings):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
        for idx, order_string in enumerate(order_strings, start=1):
            # Create a DataFrame with the order string
            df = pd.DataFrame({'Order Content': [order_string]})
            df.to_clipboard()
            
            
            
            # Write the DataFrame to the Excel sheet with a sheet name based on the index
            # df.to_excel(writer, sheet_name=f"Order_{idx}", index=False)
    
    # Save the Excel file

# Example usage with a list of order strings
order_strings_with_delimiter = [
    'Order: DF8E5AB6 Delivered The order was delivered at 10:56 PM on September 24, 2023.  Rate Dasher Get help...',
    'Order: 1D4B1698 Delivered The order was delivered at 7:18 PM on September 24, 2023.  Rate Dasher Get help...'
]

create_excel_from_orders(order_strings_with_delimiter)
